<a href="https://colab.research.google.com/github/Centrattic/AI4ALL2021-NLP/blob/master/HandPD/NeuralNets/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We are going to take text-based features and apply a simple, fully-connected neural network to them. Atually, let's use images - that's what the tutorial does. Let's only consider Meander for the time being.

In [2]:
# General
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import PIL
from PIL import Image
import pandas as pd
import os
import math

# Image Reading
import cv2
import glob

# Other ML Preprocessing
from sklearn.model_selection import train_test_split

# Torch General
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn # loss functions, neural network type (convolutional, linear, etc.)
import torch.optim as optim # optimization functions (sgd)
import torch.nn.functional as F # functions without parameters - activation functions (Relu, etc.) (also included in nn package, could use, but functional package is "better")
from torch.utils.data import DataLoader
import torchvision.datasets as datasets # torch has a LOT LOT LOT of standard datasets (ImageNet, MNIST, etc.)
import torchvision.transforms as transforms # transformations for dataset
from torch.utils.data.sampler import SubsetRandomSampler # PyTorch train test split

In [3]:
"""class NN(nn.Module):
  def __init__(self, input_size, num_classes): # input-size = 611568 (size of our images, pixel number), num_classes = 2 (PD/no PD)
    super(NN, self).__init__() # initializes the NN class that we're defining
    self.fc1 = nn.Linear(input_size, 50) # 50 nodes
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x): # run on some input x, which is the images which we run through fc1 and fc2 layers created above (and add the reLU activation function it between)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x """

"class NN(nn.Module):\n  def __init__(self, input_size, num_classes): # input-size = 611568 (size of our images, pixel number), num_classes = 2 (PD/no PD)\n    super(NN, self).__init__() # initializes the NN class that we're defining\n    self.fc1 = nn.Linear(input_size, 50) # 50 nodes\n    self.fc2 = nn.Linear(50, num_classes)\n\n  def forward(self, x): # run on some input x, which is the images which we run through fc1 and fc2 layers created above (and add the reLU activation function it between)\n    x = F.relu(self.fc1(x))\n    x = self.fc2(x)\n    return x "

Now to create the convolutional partially connected network.

Only for square image it seems? our image is not square. What exactly is input features?

same convolution keeps n out = n in

<img src="https://i.stack.imgur.com/vD1u3.png">

In [4]:
# consider how different layers change output size!!

class CNN(nn.Module): # inherit from nn.Module (parent class)
  def __init__(self, in_channels = 3, num_classes = 2): # colors, so 3 channels
    super(CNN,self).__init__()
    # Convolution Layer
    # out_channels, kernel_size, stride, padding can all be changed (we are doing our own padding, not too sure what padding is? Looky up.)
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size = (3,3), stride = (1,1), padding=(1,1))
    self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2)) # will halve the size of our features, we'll run it twice (so 744/2/2 = 186, 822/2/2 = 205.5, become 205 im guessing)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size = (3,3), stride = (1,1), padding=(1,1))
    self.fc1 = nn.Linear(16 * 186 * 205, num_classes)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1) # getting long list of features, not like a 4d tensor
    x = self.fc1(x)

    return x

### Quick Test

What the model should output it something of shape [264 (140 + 124), 2]. For each image, it should predict the probability of it being in class 1 or 2 and return both of those.

In [5]:
model = CNN() # we no longer need input_size?? Plus we've defaulted the two things we did need :D

In [6]:
# 124 patient images
# 140 control images 

# data must be like (batch, channels, width, height) or height, width - order dosnt matter for me cause rotated

x = torch.randn(64, 3, 744, 822) # is 64 our batch size?? I think so, that's what model sees one at a time! So i can't run full batch (of 264) into model. Fair. Ig I could with Linear since it was simpler.

print (model(x).shape)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([64, 2])


## Set Device + Init Hyperparams

In [7]:
# Device set 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # google colab provides cuda gpu
print (device)

cuda


In [8]:
# Hyperparams
in_channel = 3 
# No longer using input size
num_classes = 2

# tunables
test_size = 0.2
learning_rate = 0.001
batch_size = 64
num_epochs = 3

## Load Data

### Preprocessing

In [9]:
# Load Data (possible to do it with Google Drive rather than uploading from computer?).
# Uploaded can be nice tho... especially because it automatically goes to dictionary... that can be fed into NN, with file names as indexes - quite nice

""" from google.colab import files
uploaded = files.upload()"""

# %cd "/content/drive/"My Drive"/Data/Images/Meander/"
# X_train = np.load('HealthyMeander/HealthyMeander/')


' from google.colab import files\nuploaded = files.upload()'

In [10]:
# Healthy: Class 0 . PD: Class 1

def extract_images(path, c): # path of data, class of data
  filename_arr = []
  X_arr = []

  for file in glob.glob(path):
    filename_arr.append(file) # filenames, not going to use them for now. Might need them later.
    x = cv2.imread(file)
    X_arr.append(x)

  y_arr = [c] * len(X_arr)

  return X_arr, y_arr

# X_arr is a list of 3D arrays representing images

In [11]:
# possibly try different pad modes for better NN results? (instead of 'constant')
def pad_images(arr):
  arr = np.copy(arr)
  largestX = 0
  largestY = 0

  def pad_condition(pad, largest, index):
    if (2 * pad != (largest - arr[i].shape[index])):
      pad1 = pad + 1
      pad2 = pad
    else:
      pad1, pad2 = pad, pad
    return pad1, pad2

  for i in arr:
    X = i.shape[0]
    Y = i.shape[1]
    if (X > largestX):
      largestX = X
    if (Y > largestY):
      largestY = Y

  for i in range(len(arr)):
    X_pad = int((largestX - arr[i].shape[0]) /2) # pad equally in both directions, must be int
    Y_pad = int((largestY - arr[i].shape[1]) /2)
    
    # but int floors, so we might get something of a slightly wrong shape (by 1), so...
    X_pad1, X_pad2 = pad_condition(X_pad, largestX, 0)
    Y_pad1, Y_pad2 = pad_condition(Y_pad, largestY, 1)

    arr[i] = np.pad(arr[i], ((X_pad1, X_pad2), (Y_pad1, Y_pad2), (0, 0)), 'constant', constant_values=(0))

  maxSize = largestX * largestY
  
  return arr, maxSize

In [12]:
def normalize_images(arr, num):
  arr = arr/num

  for i in range(len(arr)): # reshape for right size. Currently it is of shape (744, 822, 3), or (w, h, c). I want (c, w, h)
    w = arr[i].shape[0]
    h = arr[i].shape[1]
    c = arr[i].shape[2]
    arr[i] = arr[i].reshape((c,w,h))

  return arr

In [13]:
""" X_meah, y_meah = extract_images("/content/drive/MyDrive/Data/Images/Meander/HealthyMeander/HealthyMeander/*.*",0)
X_meap, y_meap = extract_images("/content/drive/MyDrive/Data/Images/Meander/PatientMeander/PatientMeander/*.*",1)

X_all = X_meah + X_meap
X_all, maxSize = pad_images(X_all)

X_all = normalize_images(X_all, 255)

y_all = np.array(y_meah + y_meap) """

# Why the warning particularly? 

""" X_all[0].shape
X_all = np.stack(X_all, axis=0)
X_all.shape """ # YAAAY 

' X_all[0].shape\nX_all = np.stack(X_all, axis=0)\nX_all.shape '

In [14]:
class MeanderDataset(Dataset):
  def __init__(self):
    # data loading
    X_meah, y_meah = extract_images("/content/drive/MyDrive/Data/Images/Meander/HealthyMeander/HealthyMeander/*.*",0)
    X_meap, y_meap = extract_images("/content/drive/MyDrive/Data/Images/Meander/PatientMeander/PatientMeander/*.*",1)

    y_all = np.array(y_meah + y_meap)

    X_all = X_meah + X_meap
    X_all, maxSize = pad_images(X_all) # just padding all of the array's images to be the same size
    X_all = normalize_images(X_all, 255) # normalizing from 0 to 255 - 0 to 1

    X_all = np.stack(X_all, axis=0) # stacking into 4D tensor

    self.n_samples = X_all.shape[0]

    self.x = torch.from_numpy(X_all).float() # creates tensor from numpy array, making it float as expected by model
    self.y = torch.from_numpy(y_all).long() # y_all is numpy array too, making it a long as expected by model
  
  # support indexing such that dataset[i] can be used to get i-th sample
  def __getitem__(self, index):
        return self.x[index], self.y[index]

  # to return size
  def __len__(self):
    return self.n_samples

In [15]:
dataset = MeanderDataset() # Meander Dataset object

# Why the warning?

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:792: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, order=order, subok=subok, copy=True)


In [16]:
first_row = dataset[0]
feature0, label0 = first_row
print(feature0, label0)

# feature0 shape is 744, 822, 3
# label0 is just 0

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.4784, 0.4471, 0.4471],
         [0.4000, 0.3608, 0.3608,  ..., 1.0000, 1.0000, 0.9882],
         [1.0000, 1.0000, 0.9843,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.3804, 0.3412, 0.3412],
         [0.2824, 0.2431, 0.2431,  ..., 1.0000, 1.0000, 0.9882],
         [1.0000, 1.0000, 0.9843,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.2980, 0.2706, 0.2118],
         [0.1804, 0.1529, 0.0941,  ..., 0.9843, 0.9804, 0.9647],
         [0.9882, 0.9843, 0.9686,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.3255, 0.2980, 0.2392],
         [0.1804, 0.1569, 0.1059,  ..., 0.9216, 0.9176, 0.9020],
         [0.9216, 0.9176, 0.9020,  ..., 0.0000, 0.0000, 0.

In [17]:
dataset[0][0].shape

torch.Size([3, 744, 822])

In [18]:
len(dataset)

264

### DataLoader


In [19]:
# Dataloader to load whole dataset
# Shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses simultaneously, set to 0 if error occurs when loading

In [20]:
# To perform train test split, we'll use sklearn... https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

seed = 0 # just random state to start at

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _ , _ = train_test_split(
    range(len(dataset)),
    dataset.y,
    stratify=dataset.y,
    test_size=test_size,
    random_state=seed
)

In [21]:
# train_indices is indices of the training values while test_indices is indicies of the testing values. Let's split our data like such.

# generate subset based on indices
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

In [22]:
len(train_dataset), len(test_dataset)

(211, 53)

In [23]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True, num_workers =2)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True, num_workers =2)

## Initialize Network

In [24]:
model = CNN().to(device) # no params, we defaulted them :))

In [25]:
# Loss + Optimizer
criterion = nn.CrossEntropyLoss() # loss function
optimizer = optim.Adam(model.parameters(), lr = learning_rate) # optimizer function

## Train Network

In [26]:
# epochs: number of times network sees images. 1 epoch - seen all images once

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader): # parts of the train_loader, (data,targets) in tuple together, batch_idx there before
        
        # Get data to cuda (that's our device, if it's possible)
        data = data.to(device=device)
        targets = targets.to(device=device)

        # No need to reshape, we flattened it previously in the forward() part of the network || data = data.reshape(data.shape[0], -1)

        # by looking at this, we see we have 4 batches: 3 of size 64, 1 of size 19. Also, our shape is right now.
        print (data.shape)

        # forward (why called forward??) - forward propagation
        scores = model(data)
        loss = criterion(scores, targets)

        # backward (why called backward??) - backward propagation
        optimizer.zero_grad() # set all gradients to 0 for each batch so it doesnt store calculation from previous batch
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

torch.Size([64, 3, 744, 822])
torch.Size([64, 3, 744, 822])
torch.Size([64, 3, 744, 822])
torch.Size([19, 3, 744, 822])
torch.Size([64, 3, 744, 822])
torch.Size([64, 3, 744, 822])
torch.Size([64, 3, 744, 822])
torch.Size([19, 3, 744, 822])
torch.Size([64, 3, 744, 822])
torch.Size([64, 3, 744, 822])
torch.Size([64, 3, 744, 822])
torch.Size([19, 3, 744, 822])


## Check Accuracy

In [27]:
def check_accuracy(loader, model):
  """ if loader.dataset.train: # if this is true, then it is loading and checking training data
    print ('Checking accuracy on training data')
  else: 
    print ('Checking accuracy on testing data')

  Appears to be error with this code  
  """

  # initializing
  num_correct = 0
  num_samples = 0

  model.eval() # why this specifically?

  with torch.no_grad(): # so we don't have to compute gradients for accuracy
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      # no reshape once again || x = x.reshape(x.shape[0], -1)

      scores = model(x)

      # scores is size 264 * 2
      # _, is don't store this part in anything
      _, predictions = scores.max(1) # gives us index of maximum score value (max along second dimension, reason for the 1) in predictions variable each time
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

  model.train()
  acc = num_correct/num_samples
  return acc

In [28]:
# running check_accuracy on training and test set
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 66.35
Accuracy on test set: 64.15
